In [40]:
import os
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [41]:
base_dir = '/home/ankit/kaggle_comp/kaggle/chest_xray/Data/'

In [42]:
train_dir = os.path.join(base_dir, 'train')

In [43]:
# Directory with our training images
train_normal_dir = os.path.join(train_dir, 'NORMAL')
train_pneumonia_dir = os.path.join(train_dir, 'PNEUMONIA')

In [44]:
train_normal_fnames = os.listdir(train_normal_dir)
train_pneumonia_fnames = os.listdir(train_pneumonia_dir)

In [45]:
val_dir = os.path.join(base_dir, 'val')

In [46]:
# Directory with our training images
val_normal_dir = os.path.join(val_dir, 'NORMAL')
val_pneumonia_dir = os.path.join(val_dir, 'PNEUMONIA')

In [47]:
val_normal_fnames = os.listdir(val_normal_dir)
val_pneumonia_fnames = os.listdir(val_pneumonia_dir)

In [48]:
test_dir = os.path.join(base_dir, 'test')

In [49]:
# Directory with our training images
test_normal_dir = os.path.join(test_dir, 'NORMAL')
test_pneumonia_dir = os.path.join(test_dir, 'PNEUMONIA')

In [50]:
test_normal_fnames = os.listdir(test_normal_dir)
test_pneumonia_fnames = os.listdir(test_pneumonia_dir)

In [51]:
print("There are total ", len(train_normal_fnames) - 1, "Normal images in the 'Train' dataset.")
print("There are total ", len(train_pneumonia_fnames) - 1, "Pneumonia images in the 'Train' dataset.")
print("There are total ", len(val_normal_fnames) -1, "Normal images in the 'Validation' dataset.")
print("There are total ", len(val_pneumonia_fnames) -1, "Penumonia images in the 'Validation' dataset.")
print("There are total ", len(test_normal_fnames) -1, "Normal images in the 'Test' dataset.")
print("There are total ", len(test_pneumonia_fnames) -1, "Penumonia images in the 'Test' dataset.")

There are total  1341 Normal images in the 'Train' dataset.
There are total  3875 Pneumonia images in the 'Train' dataset.
There are total  8 Normal images in the 'Validation' dataset.
There are total  8 Penumonia images in the 'Validation' dataset.
There are total  233 Normal images in the 'Test' dataset.
There are total  389 Penumonia images in the 'Test' dataset.


In [52]:
train_datagen = ImageDataGenerator(rescale=1.0/255.)
val_datagen = ImageDataGenerator(rescale=1.0/255.)
test_datagen = ImageDataGenerator(rescale=1.0/255.)

In [53]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', padding = 'same', input_shape=(224, 224, 3)),
    #tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding = 'same'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding = 'same'),
    #tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding = 'same'),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.MaxPooling2D(2,2), 
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding = 'same'),
    #tf.keras.layers.Conv2D(256, (3,3), activation='relu', padding = 'same'),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding = 'same'), 
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    #tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    #tf.keras.layers.Dropout(0.5),
    #tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(128, activation='relu'), 
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')  
])

In [54]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30 (Conv2D)           (None, 224, 224, 16)      448       
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 112, 112, 32)      4640      
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 56, 56, 64)        18496     
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 28, 28, 64)       

In [55]:
model.compile(optimizer=RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics = ['accuracy'])

In [56]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=50,
                                                    color_mode= 'rgb',
                                                    class_mode='binary',
                                                    target_size=(224, 224))   

Found 5216 images belonging to 2 classes.


In [57]:
val_generator =  val_datagen.flow_from_directory(val_dir,
                                                 batch_size=50,
                                                 color_mode= 'rgb',
                                                 class_mode='binary',
                                                 target_size = (224, 224))

Found 16 images belonging to 2 classes.


In [58]:
test_generator =  test_datagen.flow_from_directory(test_dir,
                                                 batch_size=50,
                                                 color_mode= 'rgb',
                                                 class_mode='binary',
                                                 target_size = (224, 224))

Found 624 images belonging to 2 classes.


In [ ]:
history = model.fit(train_generator,
                    validation_data=test_generator,
                    steps_per_epoch=50,
                    epochs=10,
                    validation_steps=None,
                    verbose=1)

Epoch 1/10
50/50 [==============================] - 190s 4s/step - loss: 0.5342 - accuracy: 0.7725 - val_loss: 0.9187 - val_accuracy: 0.7099
Epoch 2/10
50/50 [==============================] - 168s 3s/step - loss: 0.2980 - accuracy: 0.8812 - val_loss: 0.3686 - val_accuracy: 0.8510
Epoch 3/10
50/50 [==============================] - 188s 4s/step - loss: 0.2308 - accuracy: 0.9205 - val_loss: 0.8322 - val_accuracy: 0.7628
Epoch 4/10
50/50 [==============================] - 180s 4s/step - loss: 0.1485 - accuracy: 0.9424 - val_loss: 0.5443 - val_accuracy: 0.8237
Epoch 5/10
50/50 [==============================] - 170s 3s/step - loss: 0.0939 - accuracy: 0.9647 - val_loss: 1.0037 - val_accuracy: 0.7196
Epoch 6/10
50/50 [==============================] - 170s 3s/step - loss: 0.0962 - accuracy: 0.9676 - val_loss: 1.0235 - val_accuracy: 0.8029
Epoch 7/10
50/50 [==============================] - 178s 4s/step - loss: 0.0821 - accuracy: 0.9700 - val_loss: 1.2476 - val_accuracy: 0.7452
Epoch 8/10
50

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

In [ ]:
img = mpimg.imread('/home/ankit/kaggle_comp/kaggle/chest_xray/train/PNEUMONIA/person1002_bacteria_2933.jpeg')

In [ ]:
img.shape

In [ ]:
plt.imshow(mpimg.imread('/home/ankit/kaggle_comp/kaggle/chest_xray/train/PNEUMONIA/person1002_bacteria_2933.jpeg'))

In [ ]:
for i in train_normal_fnames:
    img = mpimg.imread('/home/ankit/kaggle_comp/kaggle/chest_xray/train/NORMAL/' + tr)
    size = img.shape
    print(size)

In [ ]:
plt.imshow(mpimg.imread('/home/ankit/kaggle_comp/kaggle/chest_xray/train/PNEUMONIA/person1002_bacteria_2933.jpeg'))

In [ ]:
# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

pic_index+=8

next_normal_pix = [os.path.join(train_normal_dir, fname) 
                for fname in train_normal_fnames[ pic_index-8:pic_index]]

next_pneumonia_pix = [os.path.join(train_pneumonia_dir, fname) 
                for fname in train_pneumonia_fnames[ pic_index-8:pic_index]]

In [ ]:
for i, img_path in enumerate(next_normal_pix+next_penumonia_pix):

In [ ]:
for i, img_path in enumerate(next_normal_pix+next_penumonia_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()